In [1]:
from textblob import TextBlob, Blobber
from textblob.sentiments import NaiveBayesAnalyzer

import csv
import pandas as pd
import tqdm

import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/nivpas/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [2]:
tweets_test = pd.read_csv('data/SemEval2017-task4-test.subtask-A.english.txt', sep='\t', 
                          quoting=csv.QUOTE_NONE, usecols=[0,1,2],
                          names=['id', 'label', 'message'], 
                          index_col=0, dtype={'label': 'category'})

In [3]:
blobber = Blobber(analyzer=NaiveBayesAnalyzer())

def find_sentiment(tweet):
    sentiment = blobber(tweet).sentiment
    
    if abs(sentiment.p_pos - sentiment.p_neg) < 0.1:
        return 'neutral'
    
    if sentiment.classification == 'pos':
        return 'positive'
    
    return 'negative'

In [4]:
sentiments = []

for tweet in tqdm.tqdm(tweets_test['message']):
    sentiments.append(find_sentiment(tweet))    

100%|██████████| 12284/12284 [00:15<00:00, 797.60it/s] 


In [5]:
with open('outputs/textblob-nb-predictions.txt', 'w') as f:
    for tweet_id, pred in zip(list(tweets_test.index), sentiments):
        f.write(str(tweet_id) + '\t' + pred + '\n')

In [6]:
!perl SemEval2017_task4_test_scorer_subtaskA.pl data/SemEval2017_task4_subtaskA_test_english_gold.txt outputs/textblob-nb-predictions.txt

textblob-nb-predictions.txt	0.351	0.320	0.279	
